In [1]:
from os import getcwd
from os.path import abspath, join

import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

from src.data.load_dataset import load_kmnist
from src.models.networks import V1_mnist_RFNet, classical_RFNet
from src.models.utils import train, test

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

In [3]:
# load data
train_batch_size, train_percentage = 256, 0.999
train_loader, val_loader, test_loader = load_kmnist(train_batch_size, train_percentage)

# training params
num_epochs = 10
step_size, gamma = 2, 0.5 # lr scheduler
num_trials = 50
log_interval = 100
num_neurons = sorted(set(np.logspace(0, 3.5, 50).astype('int')))
loss_fn = F.cross_entropy

/home/bpandey/anaconda3/envs/random_features/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### V1 RFNet with optimized parameters

In [ ]:
s, f, c = 5.34, 1.965, None
lr = 0.0031485838088746586

test_v1 = {'hidden_size': [], 'mean': [], 'std': []}
for hidden_size in tqdm(num_neurons):
    accuracy = []
    for trial in range(num_trials):
        model = V1_mnist_RFNet(hidden_size, s, f, c).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

        for epoch in range(num_epochs):
            _ = train(log_interval, device, model, train_loader, optimizer, epoch, loss_fn, verbose=False)
            scheduler.step()
        accuracy.append(test(model, device, test_loader, verbose=False))
        
    test_v1['hidden_size'].append(hidden_size)
    test_v1['mean'].append(np.mean(accuracy))
    test_v1['std'].append(np.std(accuracy))

  0%|          | 0/44 [00:00<?, ?it/s]

### Classical RFNet

In [ ]:
inp_size = (1, 28, 28)
lr = 0.01922083004518646

test_classical = {'hidden_size': [], 'mean': [], 'std': []} 
for hidden_size in tqdm(num_neurons):
    accuracy = []
    for trial in range(num_trials):
        model = classical_RFNet(inp_size, hidden_size).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

        for epoch in range(num_epochs):
            _ = train(log_interval, device, model, train_loader, optimizer, epoch, loss_fn, verbose=False)
            scheduler.step()
        accuracy.append(test(model, device, test_loader, verbose=False))

    test_classical['hidden_size'].append(hidden_size)
    test_classical['mean'].append(np.mean(accuracy))
    test_classical['std'].append(np.std(accuracy))

### V1 RFNet with incompatible parameters $s=0.5$, $f=0.5$

In [ ]:
s, f, c = 0.5, 0.5, None
lr = 0.0031485838088746586

test_incompatible = {'hidden_size': [], 'mean': [], 'std': []}
for hidden_size in tqdm(num_neurons):
    accuracy = []
    for trial in range(num_trials):
        model = V1_mnist_RFNet(hidden_size, s, f, c).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

        for epoch in range(num_epochs):
            _ = train(log_interval, device, model, train_loader, optimizer, epoch, loss_fn, verbose=False)
            scheduler.step()
        accuracy.append(test(model, device, test_loader, verbose=False))
        
    test_incompatible['hidden_size'].append(hidden_size)
    test_incompatible['mean'].append(np.mean(accuracy))
    test_incompatible['std'].append(np.std(accuracy))

In [6]:
# save
test_classical = {'hidden_size': [], 'mean': [], 'std': []} 
test_incompatible = {'hidden_size': [], 'mean': [], 'std': []}
test = {'v1': test_v1, 'classical': test_classical, 'incompatible': test_incompatible}
data_dir = abspath(join(getcwd(), '../../'))
with open(data_dir + '/models/results/kmnist_clf/kmnist_clf_s=%0.2f_f=%0.2f_torch.pickle' % (s, f), 'wb') as handle:
    pickle.dump(test, handle, protocol=pickle.HIGHEST_PROTOCOL) 

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(test_v1['hidden_size'], test_v1['mean'])
plt.plot(test_classical['hidden_size'], test_classical['mean'])
plt.plot(test_incompatible['hidden_size'], test_incompatible['mean'])
plt.xlim([0, 1000])

In [5]:
list(zip(test_v1['hidden_size'], test_v1['mean']))

[(1, 14.5478),
 (2, 18.0928),
 (3, 20.6776),
 (4, 22.7376),
 (5, 25.188200000000002),
 (6, 27.474400000000006),
 (7, 28.7646),
 (8, 30.3596),
 (10, 33.647999999999996),
 (11, 35.3086),
 (13, 37.75899999999999),
 (16, 40.42979999999999),
 (19, 43.3288),
 (22, 45.967800000000004),
 (26, 49.203199999999995),
 (31, 52.480599999999995),
 (37, 55.2628),
 (43, 57.8126),
 (51, 60.735199999999985),
 (61, 63.228199999999994),
 (71, 65.7868),
 (84, 68.4658),
 (100, 71.115),
 (117, 72.53320000000001),
 (138, 74.4602),
 (163, 76.0972),
 (193, 77.97040000000001),
 (227, 79.2776),
 (268, 80.6152),
 (316, 81.9154),
 (372, 83.08139999999999)]